### Utilize the narrative to do sentiment analysis
The data to use are the complaints with only narrative and "customer disputed?" label is not null.

In [1]:
#%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [37]:
complaints = pd.read_csv("data/complaints-2019-05-16_13_17.csv")
print("There are {} complaints with narrative ".format(len(complaints_narrative)))
complaints = complaints.dropna(subset=["Consumer disputed?"])


/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 164076 complaints with narrative 


In [38]:
complaints_narrative = complaints.loc[:, ["Complaint ID","Consumer complaint narrative", "Consumer disputed?"]]
print("There are {} complaints with narrative with label".format(len(complaints_narrative)))

There are 164076 complaints with narrative with label


In [6]:
complaints_narrative.head()

,Complaint ID,Consumer complaint narrative,Consumer disputed?
21029,1535719,I have a complaint regarding the overdraft fee...,No
21030,1535563,XXXX Collections Account # XXXX is reporting o...,No
21031,1535476,I just filled this out but web site timed out ...,No
21032,1534483,This file has been relisted on my credit repor...,No
21033,1535621,We took out a HELOC XXXX years ago. We thought...,No


### Remove XXXX
The narrative contains a lot of XXXX which are covered information by govenment. We need to remove them.

In [8]:
complaints_narrative["Consumer complaint narrative"] = complaints_narrative["Consumer complaint narrative"].apply(lambda x : x.replace("XXXX", ""))


In [9]:
complaints_narrative["Consumer complaint narrative"].head()

21029    I have a complaint regarding the overdraft fee...
21030     Collections Account #  is reporting on my Equ...
21031    I just filled this out but web site timed out ...
21032    This file has been relisted on my credit repor...
21033    We took out a HELOC  years ago. We thought it ...
Name: Consumer complaint narrative, dtype: object

In [7]:
print("The raitio of complaints resulting in a dispute is {}".format(len(complaints_narrative[complaints_narrative["Consumer disputed?"] == "Yes"]) / len(complaints_narrative)))
      

The raitio of complaints resulting in a dispute is 0.21823423291645336


### Split training and test data

In [10]:
from sklearn.model_selection import train_test_split
text_train, text_test, y_train, y_test = train_test_split(complaints_narrative["Consumer complaint narrative"], complaints_narrative["Consumer disputed?"], random_state=0)

### Tokenization

In [36]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
vect = CountVectorizer()
vect.fit(complaints_narrative["Consumer complaint narrative"])

KeyboardInterrupt: 

In [31]:
print("Vocabulary size: {}".format(len(vect.vocabulary_)))
print("Vocabulary content: \n{}".format(vect.vocabulary_))

Vocabulary size: 77215
Vocabulary content: 
{'have': 32430, 'complaint': 16165, 'regarding': 55805, 'the': 66850, 'overdraft': 48498, 'fees': 27896, 'that': 66746, 'were': 74073, 'billed': 10775, 'to': 68387, 'my': 44446, 'checking': 14216, 'account': 3295, 'mychecking': 44483, 'was': 73550, 'charged': 14015, 'xxxx': 75496, 'overcharge': 48478, 'for': 29086, 'withdrawals': 74764, 'in': 34932, 'which': 74320, 'had': 31866, 'funds': 30339, 'contact': 17216, 'your': 77072, 'office': 47066, 'and': 6099, 'spoke': 63245, 'with': 74738, 'representativewho': 57162, 'credited': 18648, 'me': 41917, 'of': 46948, 'back': 9146, 'however': 33698, 'fee': 27863, 'never': 45339, 'just': 38493, 'do': 23045, 'understand': 70697, 'how': 33685, 'can': 12776, 'an': 6030, 'when': 74270, 'fundswere': 30349, 'accounts': 3380, 'contacted': 17225, 'president': 52103, 'flagstar': 28692, 'bank': 9496, 'compliant': 16272, 'pushed': 53583, 'aside': 7825, 'has': 32326, 'now': 46227, 'filed': 28233, 'writ': 75271, 'ga

In [9]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
vect = CountVectorizer(stop_words="english")
vect.fit(complaints_narrative["Consumer complaint narrative"])

print("Vocabulary size: {}".format(len(vect.vocabulary_)))

Vocabulary size: 76903


In [16]:
vect_sorted = sorted(vect.vocabulary_.items(), key = lambda x : x[1])

In [20]:
print(vect_sorted[20621:20721])


[('xxxxwhen', 20621), ('xxxxwhich', 20622), ('xxxxwith', 20623), ('xxxxx', 20624), ('xxxxxx', 20625), ('xxxxxxx', 20626), ('xxxxxxxx', 20627), ('xxxxxxxx7', 20628), ('xxxxxxxxaccount', 20629), ('xxxxxxxxaddress', 20630), ('xxxxxxxxand', 20631), ('xxxxxxxxemail', 20632), ('xxxxxxxxfax', 20633), ('xxxxxxxxi', 20634), ('xxxxxxxxis', 20635), ('xxxxxxxxloan', 20636), ('xxxxxxxxon', 20637), ('xxxxxxxxplease', 20638), ('xxxxxxxxthe', 20639), ('xxxxxxxxthey', 20640), ('xxxxxxxxthis', 20641), ('xxxxxxxxto', 20642), ('xxxxxxxxtotal', 20643), ('xxxxxxxxx', 20644), ('xxxxxxxxxx', 20645), ('xxxxxxxxxxxx', 20646), ('xxxxxxxxxxxxxxxxxxxx', 20647), ('xxxxyear', 20648), ('xxxxyears', 20649), ('xxxxyou', 20650), ('xxxxyour', 20651), ('xxxxyr', 20652), ('xxxxyrs', 20653), ('ya', 20654), ('yada', 20655), ('yahoo', 20656), ('yall', 20657), ('yanked', 20658), ('yard', 20659), ('yards', 20660), ('ye', 20661), ('yea', 20662), ('yeah', 20663), ('year', 20664), ('yearly', 20665), ('years', 20666), ('yearsxxxx',

### N-gram

In [37]:
vect = CountVectorizer(ngram_range=(1,3), min_df=5, stop_words="english")
vect.fit(complaints_narrative["Consumer complaint narrative"])

print("Vocabulary size: {}".format(len(vect.vocabulary_)))
vect_sorted = sorted(vect.vocabulary_.items(), key = lambda x : x[1])

#Print the last 100 words
print(vect_sorted[-100:])

Vocabulary size: 566368
[('zenco collects', 566268), ('zero', 566269), ('zero 00', 566270), ('zero account', 566271), ('zero attempt', 566272), ('zero balance', 566273), ('zero balance account', 566274), ('zero balance accounts', 566275), ('zero balance asked', 566276), ('zero balance called', 566277), ('zero balance closed', 566278), ('zero balance credit', 566279), ('zero balance debt', 566280), ('zero balance letter', 566281), ('zero balance loan', 566282), ('zero balance owed', 566283), ('zero balance paid', 566284), ('zero balance removed', 566285), ('zero balance said', 566286), ('zero balance sent', 566287), ('zero balance told', 566288), ('zero balance transfer', 566289), ('zero balance want', 566290), ('zero balance xx', 566291), ('zero balance xxxx', 566292), ('zero balance zero', 566293), ('zero balances', 566294), ('zero called', 566295), ('zero card', 566296), ('zero chance', 566297), ('zero communication', 566298), ('zero confidence', 566299), ('zero contact', 566300), ('

### The tokenization adopted
with stop word removed. 
set min_df = 5 to filter those occasionally appeared word, say number

In [30]:
vect = CountVectorizer(min_df=5, stop_words="english")
vect.fit(complaints_narrative["Consumer complaint narrative"])

print("Vocabulary size: {}".format(len(vect.vocabulary_)))

Vocabulary size: 20420


In [31]:
vect_sorted = sorted(vect.vocabulary_.items(), key = lambda x : x[1])
print(vect_sorted[20321:20420])

[('wrongdoings', 20321), ('wronged', 20322), ('wrongful', 20323), ('wrongfull', 20324), ('wrongfully', 20325), ('wrongly', 20326), ('wrongs', 20327), ('wrote', 20328), ('ws', 20329), ('wss', 20330), ('wt', 20331), ('wth', 20332), ('wtih', 20333), ('wu', 20334), ('wv', 20335), ('ww', 20336), ('wwas', 20337), ('www', 20338), ('wy', 20339), ('wyndham', 20340), ('wyoming', 20341), ('x2', 20342), ('x3', 20343), ('xfer', 20344), ('xferred', 20345), ('xlvi', 20346), ('xoom', 20347), ('xp', 20348), ('xt', 20349), ('xtra', 20350), ('xx', 20351), ('xxx', 20352), ('xxxx', 20353), ('xxxxxx', 20354), ('ya', 20355), ('yada', 20356), ('yahoo', 20357), ('yall', 20358), ('yanked', 20359), ('yard', 20360), ('yards', 20361), ('ye', 20362), ('yea', 20363), ('yeah', 20364), ('year', 20365), ('yearly', 20366), ('years', 20367), ('yeas', 20368), ('yell', 20369), ('yelled', 20370), ('yelling', 20371), ('yellow', 20372), ('yells', 20373), ('yelp', 20374), ('yep', 20375), ('yer', 20376), ('yes', 20377), ('yeste

In [28]:
text_array = np.array(text_train)
print(text_array[0])

I have a complaint regarding the overdraft fees that were billed to my checking account. I have a complaint regarding the overdraft fees that were billed to mychecking account. I was charged  overcharge fees for  withdrawals in which I had funds in the account. I contact your office and spoke with a representativewho credited me with  of the fees back. However, the  fee was never credited. I just do n't understand how I can billed for an overdraft fee when the fundswere in my accounts. I contacted the office of the president for Flagstar Bank and my compliant was pushed aside. Flagstar has now filed a writ of garnishmentwith my employer.


### N-gram

### Lemmatization

In [26]:
import spacy
import nltk

In [30]:
en_nlp = spacy.load('en')
doc_spacy = en_nlp(text_array[0])
print("Lemmatization:")
print([token.lemma_ for token in doc_spacy])

Lemmatization:
['-PRON-', 'have', 'a', 'complaint', 'regard', 'the', 'overdraft', 'fee', 'that', 'be', 'bill', 'to', '-PRON-', 'checking', 'account', '.', '-PRON-', 'have', 'a', 'complaint', 'regard', 'the', 'overdraft', 'fee', 'that', 'be', 'bill', 'to', 'mychecke', 'account', '.', '-PRON-', 'be', 'charge', ' ', 'overcharge', 'fee', 'for', ' ', 'withdrawal', 'in', 'which', '-PRON-', 'have', 'fund', 'in', 'the', 'account', '.', '-PRON-', 'contact', '-PRON-', 'office', 'and', 'speak', 'with', 'a', 'representativewho', 'credit', '-PRON-', 'with', ' ', 'of', 'the', 'fee', 'back', '.', 'however', ',', 'the', ' ', 'fee', 'be', 'never', 'credit', '.', '-PRON-', 'just', 'do', "n't", 'understand', 'how', '-PRON-', 'can', 'bill', 'for', 'an', 'overdraft', 'fee', 'when', 'the', 'fundswere', 'in', '-PRON-', 'account', '.', '-PRON-', 'contact', 'the', 'office', 'of', 'the', 'president', 'for', 'Flagstar', 'Bank', 'and', '-PRON-', 'compliant', 'be', 'push', 'aside', '.', 'Flagstar', 'have', 'now', 

### TfidVectorizer

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

#The X and y
text_train = complaints_narrative["Consumer complaint narrative"]
y_train = complaints_narrative["Consumer disputed?"]

pipe = make_pipeline(TfidfVectorizer(min_df = 5), LogisticRegression())
param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10]}
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


AttributeError: 'GridSearchCV' object has no attribute 'best_score'

In [14]:
print("Best cross-validation score: {:.2f}".format(grid.best_score_))

Best cross-validation score: 0.78


In [15]:
vectorizer = grid.best_estimator_.named_steps["tfidfvectorizer"]
#transform the training dataset
X_train = vectorizer.transform(text_train)
# find maximum value for each of the features over the dataset
max_value = X_train.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
#get feature names
feature_names = np.array(vectorizer.get_feature_names())

print("Features with lowest tfidf:\n{}".format(feature_names[sorted_by_tfidf[:20]]))
print("Features with highest tfidf:\n{}".format(feature_names[sorted_by_tfidf[-20:]]))


Features with lowest tfidf:
['1573' '629' '1571' '1572' '479' '1574' '1575' '1576' '632' '541' '1577'
 '5316' '1693n' '986' '1693o' 'statementknown' 'anotherviolation'
 'commentsgrant' 'falseexecution' 'substantiallybenefit']
Features with highest tfidf:
['incorect' 'sure' 'la' 'mn' '15' 'unverified' 'wasnt' 'insurances' 'bank'
 'starred' '2015' '2016' '16' 'ok' 'xx' 'transunion' 'inquiry' 'paid'
 'fradulent' 'pdf']


In [29]:
print("Features each 2000:\n{}".format(feature_names[sorted_by_tfidf[::20]]))

Features each 2000:
['1573' 'todischarge' 'injures' ... 'fcs' '2012' '16']


### The TFid vectorizer does not perform good because this is a homogeneous data on financial problems and negative reviews

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/hao/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [55]:
disputed_complaints = complaints[complaints["Consumer disputed?"] == "Yes"]

### Sentiment by Vader

In [59]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords


analyser = SentimentIntensityAnalyzer()
stop_words = set(stopwords.words('english'))
sentiment_score_list = []

i = 0
for narrative in complaints["Consumer complaint narrative"]:
    print(narrative)
    score = analyser.polarity_scores(narrative)
    print(score)
    sentiment_score_list.append(score["compound"])
    i += 1
    print(complaints.loc[disputed_complaints.index[i], ["Consumer disputed?", "Company response to consumer"]])
    if i > 50:
        break;
    """
    sentenceList = sent_tokenize(row)
    for sentence in sentenceList:
        print(analyser.polarity_scores(sentence)['compound'])
        """
print(sentiment_score_list[:20])

I have a complaint regarding the overdraft fees that were billed to my checking account. I have a complaint regarding the overdraft fees that were billed to mychecking account. I was charged XXXX overcharge fees for XXXX withdrawals in which I had funds in the account. I contact your office and spoke with a representativewho credited me with XXXX of the fees back. However, the XXXX fee was never credited. I just do n't understand how I can billed for an overdraft fee when the fundswere in my accounts. I contacted the office of the president for Flagstar Bank and my compliant was pushed aside. Flagstar has now filed a writ of garnishmentwith my employer.
{'neg': 0.057, 'neu': 0.897, 'pos': 0.046, 'compound': -0.0516}
Consumer disputed?                                  Yes
Company response to consumer    Closed with explanation
Name: 21046, dtype: object
XXXX Collections Account # XXXX is reporting on my Equifax credit file as a duplicate account. The same debt with the same open date, a

Consumer disputed?                                          Yes
Company response to consumer    Closed with non-monetary relief
Name: 21141, dtype: object
In XXXX 2015, I opened a Bank of America credit card to make a big-ticket purchase from a sporting goods store because of the incentives provided by the credit card. It was the only purchase I made with the card. 

Upon receiving my first bill, I saw that there was a late fee because Bank of America had not received a payment for the previous billing statement. I was unaware that there was a previous statement. I submitted a payment for the full purchase amount, which was posted on XXXX XXXX, 2015, prior to the XXXX XXXX, 2015 due date. On XXXX XXXX, 2015, I contacted Bank of America to close the account and to see about having the late fees etc ... removed. At that time, I was informed that the account was closed, but there was nothing that could be done about the late fees. 

I contacted Bank of America again on XXXX XXXX, 2015, be

In [36]:
complaints_narrative.iloc[26773:26774]

,Complaint ID,Consumer complaint narrative,Consumer disputed?
226195,2442574,my modification was not handled correctly and ...,No


In [20]:
text_train.head()

306660    Citi bank makes it very difficult to sign in t...
222537    I signed an agreement to purchase a new constr...
284887    Ditech keeps calling me. Telling me go pay off...
282749    I have a a unverified account from PLAZA SERVI...
360587    Thursday //2016 I received a call from a numbe...
Name: Consumer complaint narrative, dtype: object

In [30]:
y_train.iloc[0:0]

Series([], Name: Consumer disputed?, dtype: object)